In [ ]:
import numpy as np
import torch 
import torch.nn as nn
import pandas as pd
from sklearn.metrics import mean_squared_error 
from sklearn.model_selection import KFold

In [ ]:
path = '/content/drive/MyDrive/python_data/社群網路與推薦系統/hw3/data/Movielens/user_movie.dat'
# path = '/content/drive/MyDrive/python_data/社群網路與推薦系統/hw3/data/Yelp/user_business.dat'
# path = '/content/drive/MyDrive/python_data/社群網路與推薦系統/hw3/data/Douban_Book/user_book.dat'
device= 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
def get_filtered_data(path):

  names = ['user_id', 'item_id', 'rating']
  df = pd.read_csv(path, sep='\t', names=names, usecols = [i for i in range(3)])
  
  '''filtering'''
  grouped_df = df.groupby(['user_id'])['item_id'].count()
  filtered_user_id = grouped_df[grouped_df>3].index
  filtered_df = df.set_index('user_id').loc[filtered_user_id].reset_index()
  filtered_data= np.array(filtered_df)

  return filtered_data

In [ ]:
def get_mask(data):
  # data 2d array with [n, 3]
  mask = torch.zeros(size= (n_user, n_item), dtype= float, device= device)
  for info in data:
    mask[info[0]-1, info[1]-1] = 1
  return mask

In [ ]:
filtered_data = get_filtered_data(path= path)

In [ ]:
global n_user, n_item, y
n_user= filtered_data[:,0].max()
n_item= filtered_data[:,1].max()

In [ ]:
y = torch.zeros(size= (n_user, n_item), dtype= float, device= device) 
for info in filtered_data:
  y[info[0]-1, info[1]-1] = info[2]
print(y.size())

torch.Size([943, 1682])


In [ ]:
class MF(nn.Module):
  def __init__(self, n_user, n_item, k, beta):
    super(MF, self).__init__()
    self.n_user= n_user
    self.n_item= n_item
    self.beta = beta
    """parameters | .to(device) creates new tensor so that the params cant be fetched from model.parameters()"""
    # self.p = torch.nn.Parameter(torch.zeros(size= (self.n_user, k), dtype= float, device= device))
    # self.q = torch.nn.Parameter(torch.zeros(size= (self.n_item, k), dtype= float, device= device))

    self.p = nn.Linear(self.n_user, k, bias= False)
    self.q = nn.Linear(k, self.n_item, bias= False)
  def forward(self):

    # predict = torch.matmul(self.p, self.q.t())
    predict= self.q(self.p.weight.t())
    return predict

  def loss_function(self, predict, y_mask):

    mask_boolean = (y_mask > 0)

    sse = torch.sum((torch.masked_select(y, mask_boolean) - torch.masked_select(predict, mask_boolean))**2)
    p_sum = torch.sum((self.p.weight.t())**2, dim= (0,1))
    q_sum = torch.sum((self.q.weight)**2, dim= (0,1))

    return 0.5*sse + self.beta * 0.5 * (p_sum + q_sum)
  
  def RMSE(self, predict, test_mask): 
    # test_mask: [n_user, n_item] float
    # predict: [n_user, n_item] float

    test_mask = (test_mask > 0)

    return round(mean_squared_error(torch.masked_select(y, test_mask).detach().cpu(), torch.masked_select(predict, test_mask).detach().cpu(), squared= False), 2)

In [ ]:
k = 15
lr = 1e-2
beta = 1
n_epoch = 5

In [ ]:
model = MF(n_user, n_item, k=k, beta= beta).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr= lr)

In [ ]:
test_kf = KFold(n_splits=5, shuffle= True, random_state=42)
val_kf = KFold(n_splits=8, shuffle= True, random_state=42)

In [ ]:
RMSE= []
for rest_indices, test_indices in test_kf.split(filtered_data):
  rest_data = filtered_data[rest_indices]
  test_data = filtered_data[test_indices]

  for train_indices, val_indices in val_kf.split(rest_data):
    train_data= rest_data[train_indices]
    val_data= rest_data[val_indices]
    
    train_mask = get_mask(data= train_data)
    val_mask = get_mask(data= val_data)

    model.train()
    optimizer.zero_grad()

    predict = model()
    loss = model.loss_function(predict= predict, y_mask= train_mask)
    loss.backward()
    optimizer.step()
  
  test_mask = get_mask(test_data)
  test_boolean= test_mask > 0
  rmse = mean_squared_error(torch.masked_select(y.cpu(), test_boolean), torch.masked_select(predict.cpu(), test_boolean).detach().numpy(), squared=False)
  RMSE.append(rmse)
  # print(rmse)

print(np.mean(RMSE))

2.9933823315415786


In [ ]:
predict

tensor([[ 2.2734,  2.5246,  2.6664,  ..., -0.0056,  1.8282,  1.7275],
        [ 1.1893,  0.7871,  1.3439,  ...,  0.6575,  0.5843,  0.5233],
        [ 0.8211,  0.4290,  0.8311,  ...,  1.0925,  0.2789,  0.2500],
        ...,
        [ 1.8179,  1.6734,  1.9919,  ...,  0.3887,  1.0818,  1.0033],
        [ 1.9414,  1.8837,  2.1415,  ...,  0.3075,  1.2982,  1.2284],
        [ 1.9437,  2.2082,  2.4175,  ..., -0.0625,  1.6775,  1.5792]],
       device='cuda:0', grad_fn=<MmBackward>)

In [ ]:
y

tensor([[5., 3., 4.,  ..., 0., 0., 0.],
        [4., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [5., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 5., 0.,  ..., 0., 0., 0.]], device='cuda:0', dtype=torch.float64)